# Mandatory Assignment 1: Counting Words

*Zachary Neveu (s191344)* 

**This is the first of three mandatory assignments to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The assignment is to be done individually. You are under no circumstances allowed to collaborate with anyone on solving the exercises (cf. the full policy on this on the course website)**
- **You must hand in one Jupyter notebook (this notebook) with your solution**
- **The hand-in of the notebook is due 2019-10-10, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python**
- **For each question you should use exactly the cells provided for your solution**
- **You should not remove the problem statements, and you should not modify the structure of the notebook**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**

---
## Introduction
In this assignment you will build data structures for counting words in a text. Suppose you are given a very large corpus of text, and from time to time you need to count how many times a word occurs. You could write a program that searches the text from start to end every time you want to make a query, but for large texts, this will be very slow. A common way to handle this, is to preprocess the text into a data structure that contains exactly the information needed to answer specific queries like the frequency of a word.

Given a text, you should build data structures that can answer the following questions efficiently:
- How many times does a given word occur in the text? (exercise 2)
- How many times in total does a word starting with a given prefix occur? (exercise 3)

For each of these questions you should write a function that organizes data in a way (the data structure) that makes it possible to write an efficient function (the query) to answer the questions. A good data structure is one where the query is much faster than just searching the text while still is not using too much space.

You should not use any Python libraries (except `string`!) to solve the exercises. You may use build-ins like lists, dictionaries, `map`, `filter`, and so on.

You should provide implementations for the functions in this notebook. Do not change the names of the functions.

To test your programs, we will use the complete works of William Shakespeare. Please run the cells below to load the text and show a preview. You should be online before you do this. Note that a good solution will have to work for even larger texts than this.

In [2]:
import requests
text = requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt').text

In [3]:
print('{} ...'.format(text[0:1000]))

This is the 100th Etext file presented by Project Gutenberg, and
is presented in cooperation with World Library, Inc., from their
Library of the Future and Shakespeare CDROMS.  Project Gutenberg
often releases Etexts that are NOT placed in the Public Domain!!

Shakespeare

*This Etext has certain copyright implications you should read!*

<<THIS ELECTRONIC VERSION OF THE COMPLETE WORKS OF WILLIAM
SHAKESPEARE IS COPYRIGHT 1990-1993 BY WORLD LIBRARY, INC., AND IS
PROVIDED BY PROJECT GUTENBERG ETEXT OF ILLINOIS BENEDICTINE COLLEGE
WITH PERMISSION.  ELECTRONIC AND MACHINE READABLE COPIES MAY BE
DISTRIBUTED SO LONG AS SUCH COPIES (1) ARE FOR YOUR OR OTHERS
PERSONAL USE ONLY, AND (2) ARE NOT DISTRIBUTED OR USED
COMMERCIALLY.  PROHIBITED COMMERCIAL DISTRIBUTION INCLUDES BY ANY
SERVICE THAT CHARGES FOR DOWNLOAD TIME OR FOR MEMBERSHIP.>>

*Project Gutenberg is proud to cooperate with The World Library*
in the presentation of The Complete Works of William Shakespeare
for your reading for educatio

---
## Exercise 1
In this exercise you should create a helper function for parsing a text into an iterable (e.g., a list) of words. You will need this in the subsequent exercises.

You should make sure that:
- each element of the iterable is exactly one word,
- all words are lower case,
- words do not contain punctuation.

Write a function `text_to_words()` that takes as input a string and output an iterable of the words in the string.

In [4]:
import string

In [5]:
def text_to_words(text): return text.lower().split(' ')

In [6]:
# You should implement this!
def text_to_words(text):
    #table = str.maketrans(string.punctuation, " "*len(string.punctuation))
    t = text.translate(str.maketrans('', '', string.punctuation))
    return t.lower().split()

In [7]:
# Should output 'this is a test'
'` `'.join(text_to_words('THIS is, a test!'))

'this` `is` `a` `test'

In [8]:
words = text_to_words(text)

In [9]:
len(words)

900987

---
## Exercise 2
In this exercise you should create a data structure that can give an answer to the following query.
- How many times does a given word occur in the text?

You should do the following:
- Write a function `build_word_count_data_structure()` that takes as input an iterable of strings, and outputs a data structure for looking up how many times a string occurs.

- Write a function `get_word_count()` that uses your data structure to count the number of times a word occurs.

- Explain with words how your data is organized in your data structure, how your function constructs it, and how a query works. Why is it efficient?

In [16]:
# You should implement this!
def build_word_count_data_structure(words):
    counts = {}
    for w in words:
        try:
            counts[w] += 1
        except:
            counts[w] = 1
    return counts

In [17]:
ds = build_word_count_data_structure(words)

In [18]:
# You should implement this!
def get_word_count(ds, word):
    return ds[word]

In [149]:
assert get_word_count(ds, 'romeo') == sum([1 for w in words if w=='romeo'])

### Explanation
A dictionary is a very simple structure that maps between keys and values. Using words as keys and counts as values allows for an $O(1)$ runtime to get the count for a word. Creating the dictionary takes $O(N)$ time, and the dictionary requires $O(N)$ space.

---
## Exercise 3
In this exercise you should create a data structure that can give an answer to the following query.
- How many times in total does a word starting with a given prefix occur?

For example, `bar` is a prefix of `bar`, `barracuda`, and `barrier`, and the result of the query should include the sum of the number of times each of those words occur.

You should do the following:
- Write a function `build_prefix_count_data_structure()` that takes as input a list of strings, and outputs a data structure for looking up how many times a prefix occurs in words.

- Write a function `get_prefix_count()` that uses your data structure to count the number of times a prefix occurs.

- Explain with words how your data is organized in your data structure, how your function constructs it, and how a query works. Why is it efficient?

In [158]:
class Trie:
    def __init__(self, words=None, name=''):
        self.name = name
        self.children = {}
        self.nsubs = 0
        if words:
            for word in words:
                self.add(word)
        
    def add(self, word):
        assert word.startswith(self.name)
        # First check if this is the final node, if so return
        self.nsubs += 1
        if word == self.name:
            return
        if word[:len(self.name)+1] not in self.children: # add child if it doesn't exist
            #print(f"Adding child {word[:len(self.name)+1]}")
            self.children[word[:len(self.name)+1]] = Trie(name=word[:len(self.name)+1])
        self.children[word[:len(self.name)+1]].add(word)
        
    def get_sum(self, word):
        if self.name == word:
            return self.nsubs
        else:
            try:
                return self.children[word[:len(self.name)+1]].get_sum(word)
            except Exception as E:
                print(E)
                return False
            
    def __repr__(self):
        return f'{self.name} ({self.nsubs} children)'

In [159]:
def build_prefix_count_data_structure(words):
    return Trie(words)

In [160]:
def get_prefix_count(trie, prefix):
    return trie.get_sum(prefix)

In [161]:
root = Trie(words)

In [189]:
root.get_sum('and')

26840

### Testing speed with random word times (vs counting)

In [190]:
import random

In [191]:
def get_random_pre():
    pre = random.choice(words)
    return pre[:random.randint(1,len(pre))]

In [184]:
pres = [get_random_pre() for i in range(100)]

Time with Trie datastructure

In [193]:
%%time 
for pre in pres:
    root.get_sum(pre)

CPU times: user 309 µs, sys: 4 µs, total: 313 µs
Wall time: 319 µs


Time searching through unique words sums (from part 2)

In [192]:
%%time
for pre in pres:
    sum([ds[w] for w in ds.keys() if w.startswith(pre)])

CPU times: user 350 ms, sys: 3 µs, total: 350 ms
Wall time: 348 ms


Time searching from scratch in word list

In [186]:
%%time 
for pre in pres:
    sum([1 for w in words if w.startswith(pre)])

CPU times: user 7.19 s, sys: 7.63 ms, total: 7.19 s
Wall time: 7.19 s


### Explanation
_Explain your data structure with words_

A *Trie* is a tree data structure which is quite useful for storing prefix-dictionaries. The key idea behind this data structure is that if you start with the first letter of a word, each letter you add greatly narrows down the number of possible words this could be. Using this property, Using this property, one can construct a tree where all children of a node have that node's value as a prefix. For this specific problem, it is also easy enough to include a count of all descendents of each node. In a Trie, the time needed to search a node as well as the time needed to add a node are proportional to the length of the word $l_w$. This makes the time to get the number of descendents $O(l_w)$. The tradeoff for this fast access time is that creating a Trie takes $O(n_w*l_w)$ time, and $O(n_{wu})$ space, where $n_w$ is the total number of words and $n_{wu}$ is the number of unique words. This tradeoff makes a lot of sense in practice for several reasons. First, the number of unique words is often much smaller than the total number of words (3% of the size in our case). Second, Most words are quite short (see the histogram below). Having an access time that is proportional to the length of a word is then very useful.

In [143]:
len(set(words)) / len(words)

0.03145772358535695

## Trie Example Figure
![Trie](https://proxy.duckduckgo.com/iu/?u=https%3A%2F%2Fkoenig-media.raywenderlich.com%2Fuploads%2F2016%2F10%2FSwiftAlgClub_TrieData-trie-1.png&f=1&nofb=1)

In [147]:
import matplotlib.pyplot as plt
%matplotlib widget
plt.style.use('Solarize_Light2')

## Frequency of Word Lengths

In [148]:
plt.hist([len(w) for w in words], 35)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([3.89930e+04, 1.50332e+05, 1.91201e+05, 2.04690e+05, 1.11393e+05,
        7.26870e+04, 5.72980e+04, 3.49530e+04, 1.98660e+04, 1.12900e+04,
        5.00000e+03, 0.00000e+00, 2.05000e+03, 6.69000e+02, 3.64000e+02,
        1.27000e+02, 3.10000e+01, 2.00000e+01, 1.10000e+01, 4.00000e+00,
        0.00000e+00, 2.00000e+00, 2.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 2.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00]),
 array([ 1.        ,  1.91428571,  2.82857143,  3.74285714,  4.65714286,
         5.57142857,  6.48571429,  7.4       ,  8.31428571,  9.22857143,
        10.14285714, 11.05714286, 11.97142857, 12.88571429, 13.8       ,
        14.71428571, 15.62857143, 16.54285714, 17.45714286, 18.37142857,
        19.28571429, 20.2       , 21.11428571, 22.02857143, 22.94285714,
        23.85714286, 24.77142857, 25.68571429, 26.6       , 27.51428571,
        28.42857143, 29.34285714, 30.25714286, 31